In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

reading csv file

In [ ]:
df_x=pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")

In [ ]:
df_x.head()

In [ ]:
df_x.shape

In [ ]:
pd.value_counts(df_x.dtypes)

In [ ]:
# For each numerical feature compute number of unique entries
unique_values = df_x.select_dtypes(
  include="number").nunique().sort_values()

In [ ]:
# For each numerical feature compute number of unique entries
unique_values = df_x.select_dtypes(
  include="number").nunique().sort_values()

# Plot information with y-axis in log-scale
unique_values.plot.bar(logy=True, figsize=(15, 4),
                       title="Unique values per feature");

In [ ]:
df_x.info()

In [ ]:
# Check number of duplicates while ignoring the index feature
n_duplicates = df_x.drop(labels=["RowId"], axis=1).duplicated().sum()
print(f"You seem to have {n_duplicates} duplicates in your database.")

In [ ]:
columns_to_consider=df_x.drop(labels=["RowId"],axis=1).columns

In [ ]:
df_x=df_x.drop_duplicates(subset=columns_to_consider)

In [ ]:
df_x.shape

In [ ]:
df_x.isna()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
plt.imshow(df_x.isna(), aspect="auto", interpolation="nearest", cmap="gray")
plt.xlabel("Column Number")
plt.ylabel("Sample Number");

In [ ]:
import missingno as msno

msno.matrix(df_x, labels=True, sort="descending");

In [ ]:
df_x.isna().mean().sort_values().plot(
    kind="bar", figsize=(15, 4),
    title="Percentage of missing values per feature",
    ylabel="Ratio of missing values per feature");

In [ ]:
df_x = df_x.dropna(thresh=df_x.shape[0] * 0.85, axis=1)
df_x.shape

In [ ]:
df_x.plot(lw=0,
          marker=".",
          subplots=True,
          layout=(-1, 4),
          figsize=(15, 30),
          markersize=1);

In [ ]:
# Create figure object with 3 subplots
fig, axes = plt.subplots(ncols=1, nrows=3, figsize=(12, 8))

# Identify non-numerical features
df_non_numerical = df_x.select_dtypes(exclude=["number", "datetime"])

# Loop through features and put each subplot on a matplotlib axis object
for col, ax in zip(df_non_numerical.columns, axes.ravel()):
  df_non_numerical.value_counts().plot(logy=True,
          marker=".",
          lw=0,
          title="col",
          figsize=(15, 30),
          ax=ax);
plt.tight_layout()

In [ ]:
# Collect entry values of the 10 most frequent accidents
accident_ids = df_non_numerical["RowId"].value_counts().head(10).index

# Removes accidents from the 'accident_ids' list
df_x = df_x[~df_x["RowId"].isin(accident_ids)]
df_x.shape

In [ ]:
# Plots the histogram for each numerical feature in a separate subplot
df_x.hist(bins=25, figsize=(15, 25), layout=(-1, 5), edgecolor="black")
plt.tight_layout();

In [ ]:
# Collects for each feature the most frequent entry
most_frequent_entry = df_x.mode()

# Checks for each entry if it contains the most frequent entry
#df_freq = df_x.eq(most_frequent_entry.values, axis=1)

# Computes the mean of the 'is_most_frequent' occurrence
#df_freq = df_freq.mean().sort_values(ascending=False)

# Show the 5 top features with the highest ratio of singular value content
#display(df_freq.head())

# Visualize the 'df_freq' table
#df_freq.plot.bar(figsize=(15, 4));



In [ ]:
# Creates mask to identify numerical features with at least 25 unique features
cols_continuous = df_x.select_dtypes(include="number").nunique() >= 25

In [ ]:
# Create a new dataframe which only contains the continuous features
df_continuous = df_x[cols_continuous[cols_continuous].index]
df_continuous.shape

In [ ]:
import seaborn as sns

sns.pairplot(df_continuous, height=1.5,
             plot_kws={"s": 2, "alpha": 0.2});

In [ ]:
# Create a new dataframe which doesn't contain continuous features
df_discrete = df_x[cols_continuous[~cols_continuous].index]
df_discrete.shape

In [ ]:
import numpy as np

# Establish number of columns and rows needed to plot all features
n_cols = 5
n_elements = len(df_discrete.columns)
n_rows = np.ceil(n_elements / n_cols).astype("int")

# Specify y_value to spread data (ideally a continuous feature)
y_value = df_x["High"]

# Create figure object with as many rows and columns as needed
fig, axes = plt.subplots(
  ncols=n_cols, nrows=n_rows, figsize=(15, n_rows * 2.5))

# Loop through features and put each subplot on a matplotlib axis object
for col, ax in zip(df_discrete.columns, axes.ravel()):
    sns.stripplot(data=df_x, x=col, y=y_value, ax=ax,
                  palette="tab10", size=1, alpha=0.5)
plt.tight_layout();

In [ ]:
# Computes feature correlation
df_corr = df_x.corr(method="pearson")

# Create labels for the correlation matrix
labels = np.where(np.abs(df_corr)>0.75, "S",
                  np.where(np.abs(df_corr)>0.5, "M",
                           np.where(np.abs(df_corr)>0.25, "W", "")))

# Plot correlation matrix
plt.figure(figsize=(15, 15))
sns.heatmap(df_corr, mask=np.eye(len(df_corr)), square=True,
            center=0, annot=labels, fmt='', linewidths=.5,
            cmap="vlag", cbar_kws={"shrink": 0.8});

In [ ]:
#  Creates a mask to remove the diagonal and the upper triangle.
lower_triangle_mask = np.tril(np.ones(df_corr.shape), k=-1).astype("bool")

#  Stack all correlations, after applying the mask
df_corr_stacked = df_corr.where(lower_triangle_mask).stack().sort_values()

#  Showing the lowest and highest correlations in the correlation matrix
display(df_corr_stacked)